<a href="https://colab.research.google.com/github/phamquocanh149/Twitter_Sentiment_Analysis/blob/main/LSTM_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
path1 = "/content/drive/MyDrive/Colab Notebooks/TwitterSentimentAnalysis/twitter_training.csv"
path2 = "/content/drive/MyDrive/Colab Notebooks/TwitterSentimentAnalysis/twitter_validation.csv"
train_df = pd.read_csv(path1)
test_df = pd.read_csv(path2)
print(train_df.head())
print(test_df.head())

   2401  Borderlands  Positive  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

  im getting on borderlands and i will murder you all ,  
0  I am coming to the borders and I will kill you...     
1  im getting on borderlands and i will kill you ...     
2  im coming on borderlands and i will murder you...     
3  im getting on borderlands 2 and i will murder ...     
4  im getting into borderlands and i can murder y...     
   3364   Facebook Irrelevant  \
0   352     Amazon    Neutral   
1  8312  Microsoft   Negative   
2  4371      CS-GO   Negative   
3  4433     Google    Neutral   
4  6273       FIFA   Negative   

  I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣  
0

In [3]:
train_df.columns = ["col1", "col2", "label", "text"]
test_df.columns = ["col1", "col2", "label", "text"]
train_df = train_df.drop(columns=["col1", "col2"])
train_df['text'] = train_df['text'].str.lower()
test_df['text'] = test_df['text'].str.lower()
test_df = test_df.drop(columns=["col1", "col2"])
print(train_df.head())
print(test_df.head())

      label                                               text
0  Positive  i am coming to the borders and i will kill you...
1  Positive  im getting on borderlands and i will kill you ...
2  Positive  im coming on borderlands and i will murder you...
3  Positive  im getting on borderlands 2 and i will murder ...
4  Positive  im getting into borderlands and i can murder y...
      label                                               text
0   Neutral  bbc news - amazon boss jeff bezos rejects clai...
1  Negative  @microsoft why do i pay for word when it funct...
2  Negative  csgo matchmaking is so full of closet hacking,...
3   Neutral  now the president is slapping americans in the...
4  Negative  hi @eahelp i’ve had madeleine mccann in my cel...


In [4]:
print(train_df.shape)
print(test_df.shape)

(74681, 2)
(999, 2)


In [5]:
print(train_df.columns)
print(test_df.columns)

Index(['label', 'text'], dtype='object')
Index(['label', 'text'], dtype='object')


In [6]:
print(train_df.dtypes)
print(test_df.dtypes)

label    object
text     object
dtype: object
label    object
text     object
dtype: object


In [7]:
print(train_df.isnull().sum())
print(test_df.isnull().sum())

label      0
text     686
dtype: int64
label    0
text     0
dtype: int64


In [8]:
print(train_df['label'].unique())
print(test_df['label'].unique())
print("-----------------------------")
print(train_df['label'].value_counts())
print("-----------------------------")
print(test_df['label'].value_counts())

['Positive' 'Neutral' 'Negative' 'Irrelevant']
['Neutral' 'Negative' 'Positive' 'Irrelevant']
-----------------------------
label
Negative      22542
Positive      20831
Neutral       18318
Irrelevant    12990
Name: count, dtype: int64
-----------------------------
label
Neutral       285
Positive      277
Negative      266
Irrelevant    171
Name: count, dtype: int64


In [9]:
print(train_df['text'][0])

i am coming to the borders and i will kill you all,


In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Drop rows with missing values in the 'text' column BEFORE encoding labels
train_df.dropna(subset=['text'], inplace=True)
test_df.dropna(subset=['text'], inplace=True)

y_train = le.fit_transform(train_df['label'])
y_test = le.transform(test_df['label'])

In [11]:
import spacy
spacy.load('en_core_web_sm')

In [12]:
print(type(y_test[0]))

<class 'numpy.int64'>


In [13]:
pip install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 torchtext==0.18.0

In [14]:
import torch
from torchtext.transforms import VocabTransform
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer


/usr/local/lib/python3.12/dist-packages/torchtext/transforms.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.12/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.12/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and

In [15]:
tokenizer = get_tokenizer("spacy", language="en_core_web_sm")

In [16]:
train_token = train_df['text'].apply(tokenizer)
test_token = test_df['text'].apply(tokenizer)

In [17]:
print(train_token.head())
print(test_token.head())

0    [i, am, coming, to, the, borders, and, i, will...
1    [i, m, getting, on, borderlands, and, i, will,...
2    [i, m, coming, on, borderlands, and, i, will, ...
3    [i, m, getting, on, borderlands, 2, and, i, wi...
4    [i, m, getting, into, borderlands, and, i, can...
Name: text, dtype: object
0    [bbc, news, -, amazon, boss, jeff, bezos, reje...
1    [@microsoft, why, do, i, pay, for, word, when,...
2    [csgo, matchmaking, is, so, full, of, closet, ...
3    [now, the, president, is, slapping, americans,...
4    [hi, @eahelp, i, ’ve, had, madeleine, mccann, ...
Name: text, dtype: object


In [18]:
print(train_token[0])

['i', 'am', 'coming', 'to', 'the', 'borders', 'and', 'i', 'will', 'kill', 'you', 'all', ',']


In [19]:
import string
X_train = train_token.apply(lambda tokens: [token for token in tokens if token not in string.punctuation])
X_test = test_token.apply(lambda tokens: [token for token in tokens if token not in string.punctuation])

In [20]:
print(X_train[0])

['i', 'am', 'coming', 'to', 'the', 'borders', 'and', 'i', 'will', 'kill', 'you', 'all']


In [21]:
vocab = build_vocab_from_iterator((token for token in X_train), specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])

In [22]:
print(len(vocab))

40634


In [23]:
print(type(y_train[0]))

<class 'numpy.int64'>


In [24]:
transform = VocabTransform(vocab)

In [25]:
X_train = X_train.apply(transform)
X_test = X_test.apply(transform)

/usr/local/lib/python3.12/dist-packages/torch/_jit_internal.py:1358: UserWarning: The inner type of a container is lost when calling torch.jit.isinstance in eager mode. For example, List[int] would become list and therefore falsely return True for List[float] or List[str].
  warnings.warn(


In [26]:
print(X_train)

0         [3, 113, 399, 4, 2, 6654, 5, 3, 56, 428, 13, 27]
1            [3, 118, 174, 15, 159, 5, 3, 56, 428, 13, 27]
2           [3, 118, 399, 15, 159, 5, 3, 56, 1767, 13, 27]
3        [3, 118, 174, 15, 159, 41, 5, 3, 56, 1767, 13,...
4          [3, 118, 174, 189, 159, 5, 3, 47, 1767, 13, 27]
                               ...                        
74676    [26, 1935, 17, 2, 609, 11426, 7, 16, 2151, 8, ...
74677    [26, 1935, 17, 16, 2151, 4569, 11426, 8, 306, ...
74678    [26, 1935, 2, 609, 11426, 7, 16, 2151, 8, 43, ...
74679    [26, 1935, 729, 2, 609, 11426, 7, 16, 2151, 8,...
74680    [26, 36, 2, 609, 11426, 7, 16, 2151, 8, 36, 30...
Name: text, Length: 73995, dtype: object


In [27]:
length= [len(s) for s in X_train]
MAXLEN = max(length)
print(MAXLEN)


166


In [28]:
from collections import Counter
counter = Counter(length)
print(sorted(counter.items()))
MAXLEN = 60

[(0, 51), (1, 2470), (2, 1657), (3, 2141), (4, 2285), (5, 2583), (6, 2608), (7, 2604), (8, 2472), (9, 2491), (10, 2466), (11, 2542), (12, 2552), (13, 2548), (14, 2360), (15, 2496), (16, 2304), (17, 2092), (18, 1970), (19, 1941), (20, 1599), (21, 1524), (22, 1534), (23, 1437), (24, 1356), (25, 1206), (26, 1114), (27, 1079), (28, 1108), (29, 983), (30, 982), (31, 940), (32, 836), (33, 782), (34, 762), (35, 667), (36, 697), (37, 630), (38, 579), (39, 573), (40, 567), (41, 566), (42, 592), (43, 509), (44, 569), (45, 490), (46, 531), (47, 439), (48, 475), (49, 655), (50, 423), (51, 399), (52, 372), (53, 342), (54, 279), (55, 290), (56, 235), (57, 204), (58, 193), (59, 140), (60, 135), (61, 111), (62, 82), (63, 89), (64, 54), (65, 48), (66, 39), (67, 34), (68, 17), (69, 14), (70, 19), (71, 14), (72, 6), (73, 8), (74, 3), (75, 3), (76, 5), (77, 1), (78, 2), (80, 2), (81, 1), (84, 1), (95, 1), (97, 1), (98, 1), (99, 9), (105, 1), (107, 1), (151, 1), (166, 1)]


In [29]:
pad_index = vocab['<pad>']
def encode_sen(encode, pad_index, max_length = MAXLEN):

  if len(encode) < max_length:
    encode += [pad_index] * (max_length - len(encode))
  else:
    encode = encode[:max_length]
  return encode

In [30]:
X_train = X_train.apply(lambda x: encode_sen(x, pad_index))
X_test = X_test.apply(lambda x: encode_sen(x, pad_index))

In [31]:
print(X_train.iloc[0])
print(type(X_train.iloc[0][0]))

[3, 113, 399, 4, 2, 6654, 5, 3, 56, 428, 13, 27, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
<class 'int'>


In [32]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

class TextDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.tensor(X, dtype=torch.long)
    self.y = torch.tensor(y, dtype=torch.long)

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

  def __len__(self):
    return len(self.X)


In [33]:
import numpy as np

train = TextDataset(X_train, np.array(y_train, dtype=int))
test = TextDataset(X_test, np.array(y_test, dtype=int))

In [34]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader = DataLoader(train, batch_size=128, shuffle=True)
test_loader = DataLoader(test, batch_size=128, shuffle=False)

In [40]:
class LSTM(torch.nn.Module):

    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)

        self.LSTM = torch.nn.LSTM(embedding_dim,
                                 hidden_dim,
                                 batch_first=True)

        self.fc = torch.nn.Linear(hidden_dim, output_dim)


    def forward(self, text):
        # text dim: [batch size, sentence length]
        embedded = self.embedding(text)
        output, (hidden, cell) = self.LSTM(embedded)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]
        hidden.squeeze_(0)
        # hidden dim: [batch size, hidden dim]
        output = self.fc(hidden)
        return output

In [41]:
torch.manual_seed(14)
model = LSTM(input_dim=len(vocab),
            embedding_dim=128,
            hidden_dim=128,
            output_dim=4 # could use 1 for binary classification
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [42]:
import torch
import torch.nn as nn
from sklearn.metrics import classification_report
criterion = nn.CrossEntropyLoss()

def train_model(model, train_loader, valid_loader, optimizer, criterion, device, epochs=5):
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        epoch_acc = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            predictions = model(X_batch)

            loss = criterion(predictions, y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            acc = (predictions.argmax(1) == y_batch).sum().item() / len(y_batch)
            epoch_acc += acc

        avg_loss = epoch_loss / len(train_loader)
        avg_acc = epoch_acc / len(train_loader)

        # Evaluate
        val_loss, val_acc, val_labels, val_preds = evaluate(model, valid_loader, criterion, device)

        print(f"Epoch {epoch+1}: "
              f"Train Loss={avg_loss:.4f}, Train Acc={avg_acc:.4f} | "
              f"Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")

        # In classification report
        print("Validation Classification Report:")
        print(classification_report(val_labels, val_preds))

In [43]:
def evaluate(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            epoch_loss += loss.item()

            all_preds.extend(predictions.argmax(1).cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())

    avg_loss = epoch_loss / len(data_loader)
    avg_acc = (np.array(all_preds) == np.array(all_labels)).mean()
    return avg_loss, avg_acc, all_labels, all_preds

In [44]:
train_model(model, train_loader, test_loader, optimizer, criterion, DEVICE, epochs=20)


Epoch 1: Train Loss=1.3686, Train Acc=0.2982 | Val Loss=1.3731, Val Acc=0.2683
Validation Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.01      0.01       171
           1       0.27      1.00      0.42       266
           2       1.00      0.00      0.01       285
           3       0.00      0.00      0.00       277

    accuracy                           0.27       999
   macro avg       0.57      0.25      0.11       999
weighted avg       0.53      0.27      0.12       999

Epoch 2: Train Loss=1.1902, Train Acc=0.4650 | Val Loss=0.8182, Val Acc=0.6907
Validation Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.11      0.18       171
           1       0.71      0.85      0.78       266
           2       0.61      0.83      0.70       285
           3       0.79      0.75      0.77       277

    accuracy                           0.69       999
   macro avg 